# Наивный Байесовский классификатор

Самый простой, часто использующийся и при этом один из самых элегантных и эффективных алгоритмов классификации — Наивный Байесовский классификатор (НБК). Он:

* Устойчив к незначимым признакам, так как просто игнорирует их
* Быстро обучается и быстро возвращает предсказание
* Потребляет относительно небольшое число ресурсов
* Показывает один из лучших результатов в задаче анализа тональности текста

Наивный Байесовский классификатор основан на теореме Байеса о вероятности события. Наивность его заключается в предположении, что все анализируемые признаки независимы, это помогает упростить работу классификатора. На практике чаще всего бывает наоборот — все признаки так или иначе влияют друг на друга. Однако "наивный" алгоритм всё равно показывает хорошие результаты, и это главное.

Ниже мы подробнее изучим основу алгоритма - саму теорему Байеса и предположение, означающее "наивность" классификатора, а затем рассмотрим его результаты на конкретном примере с классификацией твитов.

## Принцип работы классификатора

Суть Наивной Байесовской классификации — отслеживание того, какой признак о каком классе свидетельствует. А то, какие признаки и каким образом рассматриваются, зависит от модели обучения классификатора. 

Например, в **модели Бернулли** допускаются только бинарные признаки (то есть признаки типа есть/нет). При анализе текстов эта модель будет смотреть, встречается определённое слово в тексте или нет, но не будет обращать внимание на то, сколько раз оно встречается. А в **мультиномиальной модели** признаками являются счетчики слов, то есть модель будет не только замечать наличие слова в тексте, но и считать, как много раз оно было использовано.

Чтобы модель понимала, какой текст к какому классу относить, её нужно обучить на уже размеченном по тональности корпусе твитов. В процессе обучения алгоритм анализирует данные примеры и строит наивную Байесовскую модель, которая для каждого твита возвращает вероятность его принадлежности к определённому классу, если известны значения всех признаков.


## Предположение о наивности

Для вычисления вероятности принадлежности твита к определённому классу модель должна вычислить вероятность класса самого по себе, вероятность наличия в твите всех слов-признаков и вероятность их общей принадлежности к классу.

Проблемы возникают именно на последнем шаге - подсчёте вероятности наличия в твите определённого класса всех слов-признаков. Это достаточно сложная задача, так как признаки становятся зависимы друг от друга, а "посмотреть" на их зависимость мы не можем. Мы даже не знаем, как эта зависимость выражается и есть ли она вообще.

Здесь и кроется выход: мы можем "наивно" предположить, что все признаки независимы друг от друга. Тогда для вычисления нужной вероятности надо всего лишь высчитать вероятность наличия каждого слова по отдельности в тексте определённого класса. Это сделать гораздо проще.

Любопытная вещь: хотя теоретически неправильно выдвигать произвольные предположения под настроение, в данном случае такой подход на удивление хорошо работает в реальных задачах.

## Использование наивного Байесовского алгоритма для классификации

После того, как модель обучилась на размеченнном корпусе, она может предсказывать тональность новых твитов, которых не было в корпусе для обучения.

Получив новый твит, она вычисляет вероятности его принадлежности к каждому из двух классов, а затем относит твит к тому классу, для которого вероятность была больше. Стоит отметить, что сами вероятности классификатор не сильно интересуют: гораздо важнее то, какой класс более правдоподобен для того или иного текста.

При этом для подсчёта модель использует все слова-признаки, которые были в обучающем корпусе. Но как быть со словами, не встречавшимися в нём? Ведь всем новым словам будет присвоена нулевая вероятность...

## Учет ранее не встречавшихся слов

Как мы уже говорили, модель вычисляет не точные вероятности, а только правдоподобия для определения лучшего класса. Однако, используя обучающий корпус, мы предполагаем, что данные корпуса содержат полную информацию обо всех возможных словах, просто мы не используем их.

Но разве какой угодно большой корпус твитов может содержать информацию обо всех твитах, которые когда-то написаны и будут написаны? Разумеется, нет, и появление в текстах для анализа новых слов введёт модель в ступор и заметно ухудшит точность предсказания.

Однако учесть все слова в языке невозможно, и эту проблему приходится обходить. Самый простой способ её решения - **сглаживание с прибавлением единицы (add-one smoothing)**.
Это очень простой приём, заключающийся в прибавлении единицы ко всем вхождениям всех слов. В его основе лежит разумное предположение, что даже если мы не видели данного слова в обучающем корпусе, есть шанс , что в корпусе просто не нашлось текстов с ним, и мы допускаем, что оно где-нибудь встречается пусть даже один раз. А для баланса единица прибавляется и к количеству новых признаков (чтобы избежать нулей), и к количеству старых (которое просто увеличивается на 1).

### Как построить наивный Байесовский классификатор в Python?

Наивный Байесовский классификатор уже реализован за нас в библиотеке Scikit-learn (или sklearn). Всё, что требуется - дать модели обучающие данные и данные для предсказания.

Scikit-learn предлагает 3 модели Наивного Байесовского классификатора: 

* Gaussian: предполагает, что атрибуты нормально распределены (https://ru.wikipedia.org/wiki/Нормальное_распределение).

* Multinomial: используется для дискретных атрибутов. Например, при текстовой классификации она будет считать, сколько раз слово-признак встречается в анализируемом тексте.

* Bernoulli: модель Бернулли, оперирует бинарными параметрами типа "0 или 1". Наример, классификация текстов с моделью bag of words, где атрибуты могут быть 0 (слово не встретилось) или 1 (слово встретилось).

Ниже рассмотрен пример использования Гауссовской модели наивного Байесовского классификатора.

In [3]:
#импортируем модель из библиотеки sklearn и библиотеку numpy для работы с массивами
from sklearn.naive_bayes import GaussianNB
import numpy as np

#зададим матрицу признаков обучающих объектов и вектор значений классов, к которым они относятся
x= np.array([[-3,7],[1,5], [1,2], [-2,0], [2,3], [-4,0], [-1,1], [1,1], [-2,2], [2,7], [-4,1], [-2,7]])
Y = np.array([3, 3, 3, 3, 4, 3, 3, 4, 3, 4, 4, 4])
#создаём модель
model = GaussianNB()

# обучаем модель на заданных данных
model.fit(x, Y)

#дадим модели признаки ещё двух объектов и получим предсказанные им классы
predicted= model.predict([[1,2],[3,4]])
print(predicted)

[3 4]
